In [23]:
#%reset
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import numpy as np
import os 
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.inspection import permutation_importance
from scipy.stats import loguniform
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix
from eli5.sklearn import PermutationImportance
from sklearn.metrics import f1_score, matthews_corrcoef, roc_auc_score
from Functions_AMR_gonorrhea import effective_unnecessary_threshold, get_best_hyperparameters, get_best_features, get_test_train_data, get_feature_effects, f1_mcc_score_threshold
hfont = {'fontname':'Helvetica'}

## read data 
CIP_data_no_drop = pd.read_csv("CIP_data_encode_prev_not_dropped.csv")
print(CIP_data_no_drop.columns)

ImportError: cannot import name 'get_feature_effects' from 'Functions_AMR_gonorrhea' (/Users/rem76/Documents/ML_Models/GISP_data_initial/Functions_AMR_gonorrhea.py)

In [12]:
################################ Get hyperparameters and best features for each model  ###########################
# logistic regression - random initial parameters
model_lr = LogisticRegression(class_weight = 'balanced', max_iter=4000, solver = "lbfgs", C = 0.27, penalty = 'l2')
# random forest - random initial parameters
model_rf = RandomForestClassifier(n_estimators = 171, min_samples_split = 1, min_samples_leaf=1, max_features = 'sqrt', max_depth = 89, random_state = 10)
# neural network - random parameters
model_nn = MLPClassifier(solver = 'lbfgs', activation = 'tanh', max_iter = 3000 ,hidden_layer_sizes= 12, alpha =1.291549665014884, random_state=10, learning_rate = 'adaptive' )
unfitted_models = [model_lr, model_rf, model_nn]
### Hyperparameter tuning
cv = RepeatedStratifiedKFold(n_splits=10, random_state=1) ## 10-fold cross validations
# logistic regression 
space_lr = dict()
space_lr['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
space_lr['penalty'] = ['l1', 'l2']
space_lr['C'] = np.arange(0, 100, .01)
best_hyperparameters_by_year_lr = {}
# random forest 
space_rf = dict()
space_rf['n_estimators'] = np.arange(100, 201, 1)
space_rf['max_depth'] = np.arange(1, 200, 1)
space_rf['min_samples_split'] = np.arange(1, 25, 1)
space_rf['min_samples_leaf'] = np.arange(1, 25, 1)
best_hyperparameters_by_year_rf = {}


# neural network 
space_nn = dict()
space_nn['activation'] = ['tanh', 'relu']
space_nn['alpha'] = np.logspace(-1, 1, 10)
space_nn['learning_rate'] = ['constant','adaptive']
space_nn['hidden_layer_sizes'] = [(4,), (6,), (8,), (10,), (12,), (13,), (14,)]
best_hyperparameters_by_year_nn = {}

space = [space_lr, space_rf, space_nn]
best_hyperparameters_by_year = [best_hyperparameters_by_year_lr, best_hyperparameters_by_year_rf,best_hyperparameters_by_year_nn]
## need for feature engineerig 
feature_names = ['MSM','MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'Midwest','PREV_REGION', 'PREV_CLINIC', 'DELTA_REGION', 'DELTA_CLINIC']
best_features_by_year_lr = {}
best_features_by_year_rf = {}
best_features_by_year_nn = {}
best_features_by_year = [best_features_by_year_lr, best_features_by_year_rf, best_features_by_year_nn]

imporances_all_models = pd.DataFrame(0, index = feature_names, columns=np.arange(len(test_years)*3) + 1)
indices_for_importance = [0,6,12]
### ROC by year 
ROC_by_year_rf = {}
ROC_by_year_lr = {}
ROC_by_year_nn = {}

ROC_by_year = [ROC_by_year_rf, ROC_by_year_lr, ROC_by_year_nn]
#### Loop set up 
threshold_seq = np.linspace(0,1,101)
test_years = [2005]
oversample = RandomOverSampler(sampling_strategy = 0.5,random_state=42) #need for neural network and random forest

model_types = ["Logistic_regression", "Neural_network", "Random_forest"]

i = 0
for year in test_years: 
     #get dataset
    years_train = np.array(range(year - 5, year))

    CIP_data_training_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin(years_train)]
    CIP_data_testing_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([year])]
     ## Get hyperparameters, new features, and auROC for each model type
    for model_type in range(len(model_types)):
        ## Hyperparameter tuning round 1
        test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev =  get_test_train_data(CIP_data_no_drop = CIP_data_no_drop, year = year, feature_names = feature_names, years_train = years_train, model_type = model_type)

        best_hyperparameters1 = get_best_hyperparameters(unfitted_models[model_type], cv, space[model_type], X_train, y_train)
        if model_type == 0:
            model_lr = LogisticRegression(class_weight = 'balanced', max_iter=5000, solver = best_hyperparameters1['solver'], C = best_hyperparameters1['C'], penalty = best_hyperparameters1['penalty'])
            model_fit = model_lr.fit(X_train, y_train)

        elif model_type == 1:
            model_rf = RandomForestClassifier(n_estimators = best_hyperparameters1['n_estimators'], min_samples_split = best_hyperparameters1['min_samples_split'], min_samples_leaf=best_hyperparameters1['min_samples_leaf'], max_features = 'sqrt', max_depth = best_hyperparameters1['max_depth'], random_state = 10)
            model_fit = model_rf.fit(X_train, y_train)

        else:
            model_nn = MLPClassifier(solver = 'lbfgs', activation = best_hyperparameters1['activation'], max_iter = 5000 ,hidden_layer_sizes= best_hyperparameters1['hidden_layer_sizes'], alpha =  best_hyperparameters1['alpha'], random_state=10, learning_rate =best_hyperparameters1['learning_rate'])
            model_fit = model_nn.fit(X_train, y_train)

    
        ## Feature engineering
        important_features = get_best_features(feature_names, model_fit, X_test, y_test)
        best_features_by_year[model_type].__setitem__(year, important_features) 
        imporances_all_models[i + indices_for_importance[model_type]] = get_feature_effects(feature_names, model_fit, X_test, y_test) #want it to be the correct block for each model

        # Get new tets/train data and redo hyperparameters
        test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev =  get_test_train_data(CIP_data_no_drop = CIP_data_no_drop, year = year, feature_names = important_features,years_train = years_train, model_type = model_type)
        best_hyperparameters2 = get_best_hyperparameters(unfitted_models[model_type], cv, space[model_type], X_train, y_train)

        best_hyperparameters_by_year[model_type].__setitem__(year, best_hyperparameters2) 

        ##
        if model_type == 0:
            model_lr = LogisticRegression(class_weight = 'balanced', max_iter=5000, solver = best_hyperparameters1['solver'], C = best_hyperparameters1['C'], penalty = best_hyperparameters1['penalty'])
            model_fit = model_lr.fit(X_train, y_train)

        elif model_type == 1:
            model_rf = RandomForestClassifier(n_estimators = best_hyperparameters1['n_estimators'], min_samples_split = best_hyperparameters1['min_samples_split'], min_samples_leaf=best_hyperparameters1['min_samples_leaf'], max_features = 'sqrt', max_depth = best_hyperparameters1['max_depth'], random_state = 10)
            model_fit = model_rf.fit(X_train, y_train)

        else:
            model_nn = MLPClassifier(solver = 'lbfgs', activation = best_hyperparameters1['activation'], max_iter = 5000 ,hidden_layer_sizes= best_hyperparameters1['hidden_layer_sizes'], alpha =  best_hyperparameters1['alpha'], random_state=10, learning_rate =best_hyperparameters1['learning_rate'])
            model_fit = model_nn.fit(X_train, y_train)

        model_fit_train = model_rf.fit(X_train, y_train)
        y_predict_test = model_fit.predict(X_test)
        y_predict_proba_test = model_fit.predict_proba(X_test)
 
        ROC= metrics.roc_auc_score(y_test, y_predict_test)
        ROC_by_year[model_type].__setitem__(year, ROC)
        i += 1

In [14]:
print(best_hyperparameters_by_year)

[{2005: {'solver': 'newton-cg', 'penalty': 'l2', 'C': 55.0}}, {2005: {'n_estimators': 163, 'min_samples_split': 5, 'min_samples_leaf': 14, 'max_depth': 81}}, {2005: {'learning_rate': 'adaptive', 'hidden_layer_sizes': (12,), 'alpha': 0.2782559402207124, 'activation': 'tanh'}}]


In [9]:
## Graph of important features 

0
1
2
